In [ ]:
!pip install wandb

     |████████████████████████████████| 2.0MB 5.5MB/s 
     |████████████████████████████████| 133kB 22.4MB/s 
     |████████████████████████████████| 163kB 19.5MB/s 
     |████████████████████████████████| 102kB 7.3MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=d014884e744afb053ec42aaa151fbfb333335ddcedb072a7b8f34b010b015dd9
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=7a4e43687504636a2f2800b07256831049230c5e00a30a110100ef66da04eba0
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#Question 1 (2 Marks)
Download the fashion-MNIST dataset and plot 1 sample image for each class as shown in the grid below. Use "from keras.datasets import fashion_mnist" for getting the fashion mnist dataset.

﻿

In [ ]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
label = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for i in range(10):
   wandb.init(project="cs6910-assignment01")
   wandb.run.name = "Q1_run_{}".format(i+1)
   for j in range(30):
     if y_train[j] == i:
       wandb.log({"examples": [wandb.Image(X_train[j], caption=label[i])]})
       break

# Question 2 (10 Marks)

Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible so that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

In [4]:
import numpy as np

def sigmoid(z):
  g = 1/(1+np.exp(-z))
  return g

def tanh(z):
  g = np.tanh(z)
  return g

def relu(z):
  return np.maximum(0,z)

In [5]:
def softmax(x):
  return np.exp(x)/np.sum(np.exp(x), axis = 0)

In [6]:
def initialize(initializer, size1, size2):
  if(initializer == "random"):
     W = np.random.randn(size1, size2)
     return W

  if(initializer == "Xavier"):
     W = np.random.randn(size1, size2) * np.sqrt(1/size2)
     return W

  print("Enter the name of initializer correctly")

In [7]:
def linear_forward(H, W, b):
  W = np.asarray(W)
  H = np.reshape(H,(H.shape[0],-1))
  A = np.matmul(W,H) + b 
  cache = (H, W, b)
  
  return A, cache

In [8]:
def activation_forward(H_prev, W, b, activation):
  A, linear_cache = linear_forward(H_prev, W, b)
  activation_cache = A
  if activation == 'relu':
    H = relu(A)
  elif activation == 'sigmoid':
    H = sigmoid(A)
  elif activation =='tanh':
    H = tanh(A)
  elif activation == 'softmax':
    H = softmax(A)

  return H, activation_cache, linear_cache

In [9]:
def initialize_Wb_matrix(X, num_hidden, size_hidden, initializer):
  layer_dims = [X.shape[0]]
  for l in range(0, num_hidden):
    layer_dims.append(size_hidden)
  layer_dims.append(10)   
  np.random.seed(3)
  Wb_matrix = {}
  update = {}
  L = len(layer_dims)            # number of layers in the network

  for l in range(1,L):
    Wb_matrix['W' + str(l)] = initialize(initializer, layer_dims[l], layer_dims[l-1])
    update['W' + str(l)] = np.zeros((layer_dims[l], layer_dims[l-1]))
    Wb_matrix['b' + str(l)] = np.zeros((layer_dims[l], 1))
    update['b' + str(l)] = np.zeros((layer_dims[l], 1))
  return Wb_matrix, update

In [10]:
def forward_propagation(X, Wb_matrix, activation):
    H = X
    L = int((len(Wb_matrix)/2))
    A_caches = []
    H_caches = [H]
    for l in range(1, L):
        H_prev = H 
        H, A_cache, linear_cache = activation_forward(H_prev, Wb_matrix['W{:d}'.format(l)], Wb_matrix['b{:d}'.format(l)], activation)
        A_caches.append(A_cache)
        H_caches.append(H)
    HL, AL, linear_cache = activation_forward(H, Wb_matrix['W%d' % L], Wb_matrix['b%d' % L], activation='softmax')
    A_caches.append(AL)
    H_caches.append(HL)
    return HL, H_caches, A_caches

# Question 3 (18 Marks)

Implement the backpropagation algorithm with support for the following optimisation functions 

- sgd
- momentum based gradient descent
- nesterov accelerated gradient descent
- rmsprop
- adam
- nadam

(12 marks for the backpropagation framework and 2 marks for each of the optimisation algorithms above)

We will check the code for implementation and ease of use (e.g., how easy it is to add a new optimisation algorithm such as Eve). Note that the code should be flexible enough to work with different batch sizes.

In [11]:
def one_hot_encoding(l, L):
  import numpy
  e = []
  for i in range(L):
    if i == l:
      e.append(1)
    else:
      e.append(0)
  e_y = np.asarray(e)
  return e_y

In [12]:
def deriv_activ(z, activation):
  if(activation == "relu"):
    x[x<=0] = 0
    x[x>0] = 1
    return x

  elif(activation == "sigmoid"):
    g_deriv = sigmoid(z) * (1 - sigmoid(z))
    return g_deriv
  
  elif(activation == "tanh"):
    deriv = 1 - (tanh(z)) ** 2
    return deriv

In [36]:
def back_propagation(H_caches, A_caches, Wb_matrix, y_hat, y, activation):
  e_y = []
  L = int((len(Wb_matrix)/2))
  num_train = y_hat.shape[1]
  num_classes = y_hat.shape[0]
  for j in range(len(y)):
    e_y.append(one_hot_encoding(y[j],num_classes))
  e_y = np.reshape(e_y,(num_classes,num_train))
  da = -(e_y - y_hat)
  grads = {}
  
  for k in reversed(range(1,L+1)):
    dW = np.matmul(da, np.transpose(H_caches[k-1]))
    db = da
    if k != 1:
      dh = np.dot(Wb_matrix['W{:d}'.format(k)].T, da)
      da = np.multiply(dh, deriv_activ(A_caches[k-2], activation))
    grads["dW" + str(k)] = dW
    grads["db" + str(k)] = db
  
  return grads

In [14]:
def sgd(Wb_matrix, grads, eta,L):
  for l in range(L):
    Wb_matrix["W" + str(l+1)] = Wb_matrix["W" + str(l+1)] - eta * grads["dW" + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - eta * grads["db" + str(l+1)]
  return Wb_matrix

def momentum(Wb_matrix, grads, eta, gamma, update,L):
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)]) + eta * grads['dW' + str(l+1)]
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)]) + eta * grads["db" + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]
  
  return Wb_matrix, update

def nag(Wb_matrix,eta,gamma, update, L):
  Wb_matrix_temp= {}
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)])
    Wb_matrix_temp['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)])
    Wb_matrix_temp["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]

  return Wb_matrix_temp, update

def rmsprop(Wb_matrix, eta, update ,beta1, eps, grads, L):
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]**2
    update["b" + str(l+1)] = beta1 * update["b" + str(l+1)] + (1-beta1) * grads["db" + str(l+1)]**2
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/np.sqrt(update['W' + str(l+1)]+eps)) * grads['dW' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/np.sqrt(update['b' + str(l+1)]+eps)) * grads['db' + str(l+1)]
  return Wb_matrix, update
  
def adam(Wb_matrix, eta, beta1, beta2, update, v, eps, grads, L,i):
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]**2
    update["b" + str(l+1)] = beta1 * update["b" + str(l+1)] + (1-beta1) * grads["db" + str(l+1)]**2

    v['W' + str(l+1)] = beta1 * v['W' + str(l+1)] + (1-beta2) * grads['dW' + str(l+1)]**2
    v["b" + str(l+1)] = beta1 * v["b" + str(l+1)] + (1-beta2) * grads["db" + str(l+1)]**2

    m_hat['W' + str(l+1)] = update['W' + str(l+1)]/(1-math.pow(beta1,i+1))
    m_hat['b' + str(l+1)] = update['b' + str(l+1)]/(1-math.pow(beta1,i+1))

    v_hat['W' + str(l+1)] = v['W' + str(l+1)]/(1-math.pow(beta2,i+1))
    v_hat['b' + str(l+1)] = v['b' + str(l+1)]/(1-math.pow(beta2,i+1))

    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/np.sqrt(v_hat['W' + str(l+1)]+eps)) * m_hat['W' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/np.sqrt(v_hat['b' + str(l+1)]+eps)) * m_hat['b' + str(l+1)]

  return Wb_matrix, update, v

In [ ]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot
import numpy as np
import math
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train_flatten = np.reshape(X_train,(X_train.shape[1]*X_train.shape[2],X_train.shape[0]))
num_train = X_train.shape[0]
max_epoch = 1;
learning_rate = 0.001
batch_size = 16
num_hidden = 3
L = num_hidden+1
size_hidden = 32
weight_decay = 0.5
initializer = 'random'
activation = 'sigmoid'
optimizer = 'nadam'
beta1 = 0.9
beta2 = 0.999
eps = 1e-8
gamma = 0.9


Wb_matrix, update = initialize_Wb_matrix(X_train_flatten, num_hidden , size_hidden ,initializer)
v = m_hat = v_hat = update

for i in range(max_epoch):

  for j in range(num_train):
    print ("Epoch",i+1, 'Training set', j+1)
    B = np.reshape(X_train_flatten[:,j], (-1, 1))   
    y_hat, H_caches, A_caches = forward_propagation(B, Wb_matrix ,activation)
    
    if (j+1)%batch_size == 0  :
      if optimizer == 'nag':
        Wb_matrix, update = nag(Wb_matrix, learning_rate,gamma, update, L)
      if optimizer == 'nadam':
        Wb_matrix, update = nag(Wb_matrix, learning_rate,gamma, update, L)

    grads = back_propagation(H_caches, A_caches, Wb_matrix, y_hat, [y_train[j]], activation)
    
    if (j+1)%batch_size == 0 : 
      if optimizer == 'sgd':
        Wb_matrix = sgd(Wb_matrix, grads, learning_rate,L)
      elif optimizer == 'momentum':
        Wb_matrix,update = momentum(Wb_matrix,grads,learning_rate,gamma,update,L)
      elif optimizer == 'nag':
        Wb_matrix,update = momentum(Wb_matrix,grads,learning_rate,gamma,update,L)
      elif optimizer == 'rmsprop':
        Wb_matrix, update = rmsprop(Wb_matrix, learning_rate, update,  beta1, eps, grads, L)
      elif optimizer == 'adam':
        Wb_matrix, update,v = adam(Wb_matrix, learning_rate, beta1, beta2, update, v, eps, grads, L,i)
      elif optimizer == 'nadam':
        Wb_matrix, update,v = adam(Wb_matrix, learning_rate, beta1, beta2, update, v, eps, grads, L,i)
  



  



    
      


Epoch 1 Training set 1
Epoch 1 Training set 2
Epoch 1 Training set 3
Epoch 1 Training set 4
Epoch 1 Training set 5
Epoch 1 Training set 6
Epoch 1 Training set 7
Epoch 1 Training set 8
Epoch 1 Training set 9
Epoch 1 Training set 10
Epoch 1 Training set 11
Epoch 1 Training set 12
Epoch 1 Training set 13
Epoch 1 Training set 14
Epoch 1 Training set 15
Epoch 1 Training set 16
Epoch 1 Training set 17
Epoch 1 Training set 18
Epoch 1 Training set 19
Epoch 1 Training set 20
Epoch 1 Training set 21
Epoch 1 Training set 22
Epoch 1 Training set 23
Epoch 1 Training set 24
Epoch 1 Training set 25
Epoch 1 Training set 26
Epoch 1 Training set 27
Epoch 1 Training set 28
Epoch 1 Training set 29
Epoch 1 Training set 30
Epoch 1 Training set 31
Epoch 1 Training set 32
Epoch 1 Training set 33
Epoch 1 Training set 34
Epoch 1 Training set 35
Epoch 1 Training set 36
Epoch 1 Training set 37
Epoch 1 Training set 38
Epoch 1 Training set 39
Epoch 1 Training set 40
Epoch 1 Training set 41
Epoch 1 Training set 42
E

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


Streaming output truncated to the last 5000 lines.
Epoch 1 Training set 50264
Epoch 1 Training set 50265
Epoch 1 Training set 50266
Epoch 1 Training set 50267
Epoch 1 Training set 50268
Epoch 1 Training set 50269
Epoch 1 Training set 50270
Epoch 1 Training set 50271
Epoch 1 Training set 50272
Epoch 1 Training set 50273
Epoch 1 Training set 50274
Epoch 1 Training set 50275
Epoch 1 Training set 50276
Epoch 1 Training set 50277
Epoch 1 Training set 50278
Epoch 1 Training set 50279
Epoch 1 Training set 50280
Epoch 1 Training set 50281
Epoch 1 Training set 50282
Epoch 1 Training set 50283
Epoch 1 Training set 50284
Epoch 1 Training set 50285
Epoch 1 Training set 50286
Epoch 1 Training set 50287
Epoch 1 Training set 50288
Epoch 1 Training set 50289
Epoch 1 Training set 50290
Epoch 1 Training set 50291
Epoch 1 Training set 50292
Epoch 1 Training set 50293
Epoch 1 Training set 50294
Epoch 1 Training set 50295
Epoch 1 Training set 50296
Epoch 1 Training set 50297
Epoch 1 Training set 50298
Epoc

In [ ]:
import numpy as np
np.array([1,2])